# K-Means Clustering (N-Optimized Version)

This is a version of k-means algorithm in which it is precently applied an optimization technique. Basically this code performs a k-means to cluster faces. Each cluster is precedently sub-clustered in order to perform a better precision. The number of subclusters is choosen dynamically by the algorithm itself.

##Optimization
Before clustering, the dataset labelled to each subject is indipendently subclustered in n subclusters to increase the precision. This method is meant to reduce the number of elements per cluster in order to increase their spacial intercorrelation.

##Optimum number of sub-clusters
The optimum number of n is determined using elbow method within the elements of each clusters. This approach is characterized by a conservative approach, preferring an inferior number of sub-clusters in case of doubt situations 

##The code
The code performs all the operations to prepare the clustering and the operations needed to pair the generated labels of the clustering with the old ones.

##The testing
Each subject has an old_label. That old_label is linked to other 2 labels. Each label is linked to one generated_label by the array solution.

There are 2 ways to test the clustering. The first one starts from the subject to identify returning the corresponding labels. The second one starts from a testing file, returning the precision of the labelling operation.

#Code

##Setup

In [1]:
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
import csv

import math
import matplotlib.pyplot as plt
%matplotlib inline

import random
from sklearn.decomposition import PCA

from pulp import *

In [2]:
#Dataset
url = '../csv_files/final_training.csv' 

dataset = []

#Read csv and put everything into a matrix
reader = csv.reader(open(url), delimiter=",")
x = list(reader)
dataset = np.array(x).astype("float")

#Sort the dataset matrix to ensure consistent calculation
row_length = dataset[0,:].size
dataset = sorted(dataset,key=lambda x:x[row_length-1])
dataset = np.array(dataset)

#Variables

#Setting variables
labels = dataset[:,(dataset.shape[1]-1)]
dataset = dataset[:,:(dataset.shape[1]-1)]
dataset_size = dataset[:,0].size
row_length = dataset[0,:].size

#Number of clusters
number_clusters = 1
for i in range(1, dataset_size):
  if labels[i] != labels[i-1]:
    number_clusters += 1
    
#Labels without repetitions
labels_h = []
labels_h.append(labels[0])
for i in range(1, dataset_size):
  if (labels[i] != labels[i-1]):
    labels_h.append(labels[i])
    
old_labels = labels
old_labels_h = labels_h
    
print('dataset_size:', dataset_size)
print('number_clusters:', number_clusters)
print('labels_h:', labels_h)

dataset_size: 4320
number_clusters: 62
labels_h: [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0]


##Optimization

In [3]:
#Max number of iterations
#To avoid to go have too many clusters for time wasting
max_subcluster = 5

number_subclusters = 0
isOptimum = 0
# Used to determine the correct number of subclusters
# It has to be minimized
old_elbow_sum = 1000000

while isOptimum == 0 and number_subclusters < max_subcluster:
  number_subclusters += 1
  
  variances = []
  #Subclustering and generation of new labels
  for i in range(number_clusters):
    curr_cluster = []

    #Get the current label rows and their indexes
    for j in range(dataset_size):
      if old_labels[j] == old_labels_h[i]:
        curr_cluster.append(dataset[j])
      curr_cluster_size = len(curr_cluster)

    kmeans = KMeans(n_clusters=number_subclusters, random_state=0).fit(curr_cluster)
    generated_labels = kmeans.labels_

    #Validation of the subclusters
    for l in range(number_subclusters):
      curr_subcluster = []

      #Get the subcluster
      for q in range(curr_cluster_size):
        if generated_labels[q] == l:
          curr_subcluster.append(curr_cluster[q])
      curr_subcluster_size = len(curr_subcluster)
      
      if curr_subcluster_size > 0:
        #Get the centroid of all the points of the cluster
        centroid = []
        for q in range(row_length):
          addend = 0
          for s in range(curr_subcluster_size):
            addend += curr_subcluster[s][q]
          addend = addend / curr_subcluster_size
          centroid.append(addend)

        #Calculating the distances from the centroid
        distances = []
        for q in range(curr_subcluster_size):
          addend = 0
          for s in range(row_length):
            addend += (curr_subcluster[q][s] - centroid[s]) * (curr_subcluster[q][s] - centroid[s])

          distances.append(math.sqrt(addend))

        distances_size = len(distances)

        #Get the average mean
        avg_mean = 0
        for q in range(distances_size):
          avg_mean += distances[q]
        avg_mean = avg_mean/distances_size

        #Get the variance
        variance = 0
        for q in range(distances_size):
          variance += (distances[q] - avg_mean) * (distances[q] - avg_mean)
        variance = variance / distances_size
        variances.append(variance)
      
  #Check if the optimum criteria is met. Elbow method
  elbow_sum = 0
  for l in range(number_subclusters):
    elbow_sum += variances[l]
  
  #Check if it is optimum
  if old_elbow_sum <= elbow_sum :
    isOptimum = 1
    number_subclusters -= 1
  else:
    old_elbow_sum = elbow_sum

print('optimum number of subcluster:', number_subclusters)

#Subclustering in the proper number of subcluster
for i in range(number_clusters):
  curr_cluster = []
  curr_cluster_index = []

  #Get the current label rows and their indexes
  for j in range(dataset_size):
    if old_labels[j] == old_labels_h[i]:
      curr_cluster.append(dataset[j])
      curr_cluster_index.append(j)

      #Pre updating labels
      labels[j] = old_labels[j] * 1000

  if len(curr_cluster) >= (number_subclusters):
    #subclustering
    kmeans = KMeans(n_clusters=number_subclusters, random_state=0).fit(curr_cluster)
    generated_labels = kmeans.labels_

    for j in range(generated_labels.size):
      labels[curr_cluster_index[j]] = old_labels[curr_cluster_index[j]] + generated_labels[j]

optimum number of subcluster: 3


In [4]:
#Putting the optimized dataset back together
dataset = np.insert(dataset, dataset.shape[1], labels, axis=1)

In [5]:
#Sort the dataset matrix for further elaboration
dataset = sorted(dataset,key=lambda x:x[row_length])
dataset = np.array(dataset)

In [6]:
#Setting variables on optimized dataset
labels = dataset[:,(dataset.shape[1]-1)]
dataset = dataset[:,:(dataset.shape[1]-1)]
dataset_size = dataset[:,0].size
row_length = dataset[0,:].size

#Number of clusters
number_clusters = 1
for i in range(1, dataset_size):
  if labels[i] != labels[i-1]:
    number_clusters += 1
    
#Labels without repetitions
labels_h = []
labels_h.append(labels[0])
for i in range(1, dataset_size):
  if (labels[i] != labels[i-1]):
    labels_h.append(labels[i])
    
print('dataset_size:', dataset_size)
print('number_clusters:', number_clusters)
print('labels_h:', labels_h)

dataset_size: 4320
number_clusters: 186
labels_h: [1000.0, 1001.0, 1002.0, 2000.0, 2001.0, 2002.0, 3000.0, 3001.0, 3002.0, 4000.0, 4001.0, 4002.0, 5000.0, 5001.0, 5002.0, 6000.0, 6001.0, 6002.0, 7000.0, 7001.0, 7002.0, 8000.0, 8001.0, 8002.0, 9000.0, 9001.0, 9002.0, 10000.0, 10001.0, 10002.0, 11000.0, 11001.0, 11002.0, 12000.0, 12001.0, 12002.0, 13000.0, 13001.0, 13002.0, 14000.0, 14001.0, 14002.0, 15000.0, 15001.0, 15002.0, 16000.0, 16001.0, 16002.0, 17000.0, 17001.0, 17002.0, 18000.0, 18001.0, 18002.0, 19000.0, 19001.0, 19002.0, 20000.0, 20001.0, 20002.0, 21000.0, 21001.0, 21002.0, 22000.0, 22001.0, 22002.0, 23000.0, 23001.0, 23002.0, 24000.0, 24001.0, 24002.0, 25000.0, 25001.0, 25002.0, 26000.0, 26001.0, 26002.0, 27000.0, 27001.0, 27002.0, 28000.0, 28001.0, 28002.0, 29000.0, 29001.0, 29002.0, 30000.0, 30001.0, 30002.0, 31000.0, 31001.0, 31002.0, 32000.0, 32001.0, 32002.0, 33000.0, 33001.0, 33002.0, 34000.0, 34001.0, 34002.0, 35000.0, 35001.0, 35002.0, 36000.0, 36001.0, 36002.0, 3700

##Clustering Block

In [7]:
#Applying kmeans classifier
kmeans = KMeans(n_clusters=number_clusters, random_state=0).fit(dataset)
generated_labels = kmeans.labels_

In [8]:
#Filling this array before doing the validation
generated_labels_h = [] 

#Copy the list of the labels to avoid side effects
sorted_generated_labels = generated_labels.copy()
sorted_generated_labels.sort()

#Extract generated labels without repetition
generated_labels_h.append(sorted_generated_labels[0])
for i in range(1, dataset_size):
  if (sorted_generated_labels[i] != sorted_generated_labels[i-1]):
    generated_labels_h.append(sorted_generated_labels[i])

##Matching Block

In [9]:
#Building the flow matrix
flow = []    

for curr_label in range(number_clusters):
  #Getting the precision flow per label index
  precision_flow = np.zeros(number_clusters)
  
  #Getting the generated labels for the current labels
  generated_curr_labels = []
  for i in range(dataset_size):
    if labels[i] == labels_h[curr_label]:
      generated_curr_labels.append(kmeans.predict([dataset[i,:]]))

  generated_curr_labels.sort()
  curr_generated_label = generated_curr_labels[0]
  curr_generated_label_occurrences = 1
  
  for i in range(1, len(generated_curr_labels)):
    
    if curr_generated_label != generated_curr_labels[i]:
      #Se la label cambia si calcola la percentuale nel vettore indicizzato dalla label
      precision_flow[curr_generated_label] = curr_generated_label_occurrences / len(generated_curr_labels) * 100
      
      curr_generated_label = generated_curr_labels[i]
      curr_generated_label_occurrences = 1
    else:
      #If the label doesn't change, it is set one occurrence more
      curr_generated_label_occurrences += 1
  
  #Otherwise the algorithm doesn't find the last item because there is no
  #other element at the end to trigger the if condition
  precision_flow[curr_generated_label] = curr_generated_label_occurrences / len(generated_curr_labels) * 100
  
  flow.append(precision_flow)

In [10]:
def calculate_matching(flow):
  n = flow[0].size #number of subjects = number of clusters
  problem = LpProblem("From Label to Cluster", LpMinimize)
  
  #GENERATE LP PROBLEM VARIABLES
  x = [[]]*n
  for i in range(0, n):
    x[i] = np.empty(n, dtype=LpVariable)
    for j in range(0, n):
      #generate variable which represents flux on edge (u, v) for all u representing labels and for all v representing clusters
      exec("x[%d][%d] = LpVariable(\"x_%d_%d\",0,1,LpInteger)" % (i, j, i, j))      
  
  #SET THE OBJECTIVE FUNCTION
  problem += sum((-flow[i][j])*x[i][j] for i in range(0, n) for j in range(0, n)), "Objective function"
  
  #COSTRAINTS
  problem += sum(x[i][j] for i in range(0, n) for j in range(0, n)) == n, "Exactly n couples found"
  
  #COSTRAINTS
  for i in range(0, n):
    str = "Exactly one cluster for label %d" %i
    problem += sum(x[i][j] for j in range(0, n)) == 1, str
    
  #COSTRAINTS
  for j in range(0, n):
    str = "Exactly one label for cluster %d" %j
    problem += sum(x[i][j] for i in range(0, n)) == 1, str
    
  #SOLVE
  matching = [0]*n
  #print(problem)
  if problem.solve() == 1: #OPTIMUM OBTAINED: SET RETURN VALUES
    for v in problem.variables():
      if v.varValue == 1:
        split = v.name.split("_")
        i = int(split[1])
        j = int(split[2])
        #uncomment to return an array where the i-th value is our custom label corresponding to the i-th cluster
        #matching[j] = i
        #uncomment to return an array where the i-th value is the cluster corresponding to the i-th subject
        matching[i] = j
        
  else: 
    print("Matching not found!")
    
  return matching

In [11]:
#The array 'solution' links the index of the starting label
#with the generated one. For example
# let labels_h = [5, 7, 4, 2]
# so solution[2] means the generatedd label corresponding to 
# the originary label 4
solution = calculate_matching(flow)

##Validation Block

In [ ]:
#Set the overall precision of the model
precision = []
overall = 0
for i in range(number_clusters):
  precision.append(flow[i][solution[i]])
  
  #Number of items per cluster
  number_items = 0
  for j in range(dataset_size):
    if labels_h[i] == labels[j]:
      number_items += 1
      
  overall += (precision[i] * number_items)

overall = overall / dataset_size
print(overall, '%')

##Plotting Block

In [ ]:
#Normalizing data for visualization
dataset_norm = (dataset - dataset.min())/(dataset.max() - dataset.min())

pca = PCA(n_components=2)
transformed = pd.DataFrame(pca.fit_transform(dataset_norm))

plt.scatter(transformed[generated_labels==i][0], transformed[generated_labels==i][1], label=i)
  
  
plt.legend()
plt.show()

##Testing Block

In [ ]:
subject = 10
#Manual checking
subject_label_index = [] # the indexes in labels_h of the subclustered labels
subject_labels = [] # the subclustered label_h - for visualization only

#Extraction of the originary labels belonging to the subject
for i in range(number_clusters):
  if old_labels_h[subject]*1000 <= labels_h[i]:
    if (old_labels_h[subject]+1)*1000 > labels_h[i]:
      subject_label_index.append(i)
      subject_labels.append(labels_h[i]) #for visualization only

error = 0
num_items = 0
subject_generated_labels = []  # for visualization only
subject_correct_generated_labels = [] # for visualization only

#Getting the precision
for i in range(len(subject_label_index)):
  curr_label = labels_h[subject_label_index[i]]
  curr_solution = solution[subject_label_index[i]]
  
  for j in range(dataset_size):
    if labels[j] == curr_label:
      tested_label = kmeans.predict([dataset[j,:]])
      subject_generated_labels.append(tested_label[0]) # for visualization only
      subject_correct_generated_labels.append(curr_solution) # for visualization only
      num_items += 1
      
      if tested_label != curr_solution:
        error += 1

#Printing
print('Subject number:', subject)
print('Dataset label:',old_labels_h[subject])
print('Associated labels:',subject_labels)
print()
print('Solution labels:',subject_correct_generated_labels)
print('Predicted labels:',subject_generated_labels)
print('Precision:', (num_items-error)/num_items*100, '%')

In [ ]:
#Test from dataset
url = "../csv_files/testing/1_Indoor_lights_cooperative_testing.csv"

test_dataset = []

#Read csv and put everything into a matrix
reader = csv.reader(open(url), delimiter=",")
x = list(reader)
test_dataset = np.array(x).astype("float")

#Variables of the testing dataset
test_labels = test_dataset[:,(test_dataset.shape[1]-1)]
test_dataset = test_dataset[:,:(test_dataset.shape[1]-1)]
test_dataset_size = test_dataset[:,0].size

#Predict
tested_predicted_labels = kmeans.predict(test_dataset)

correct_matchings = 0
wrong_matchings = 0

for i in range(test_dataset_size):
  
  for j in range(number_clusters):
    if solution[j] == tested_predicted_labels[i]:
      paired_sublabel = labels_h[j]
      originary_label = int(round(labels_h[j]/1000))
      
      if (test_labels[i] == originary_label):
        correct_matchings += 1
      else:
        wrong_matchings += 1
        
print('Correct Matchings:', correct_matchings)
print('Wrong Matchings:', wrong_matchings)
print('Precision: ', correct_matchings / (correct_matchings + wrong_matchings) * 100, '%')